<a href="https://colab.research.google.com/github/hdperezn/EEG_PianoRoll/blob/main/EEGNet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auxiliar functions

In [1]:
from google.colab.drive import mount
mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip install scikeras[tensorflow] >/dev/null
!dir

drive  sample_data


In [3]:
from scipy.signal import resample
from sklearn.metrics import accuracy_score, cohen_kappa_score, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score
from imblearn.under_sampling import NearMiss
from os import makedirs

import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
import numpy as np
import pandas as pd
import pickle
from tensorflow import constant
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.manifold import TSNE

In [4]:
!git clone https://github.com/hdperezn/EEG_PianoRoll.git

Cloning into 'EEG_PianoRoll'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 5), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (19/19), 7.95 KiB | 7.95 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [5]:
from EEG_PianoRoll.functions.networks import EEGNet
from EEG_PianoRoll.functions.data_processing import segmentation_trials, kappa

In [6]:
model_dir = 'models/'
makedirs(model_dir)

# Accuracy Multiclass

In [9]:
path_DEAP = '/content/drive/Shareddrives/GCPDS-Databases/DEAP/'
save_models_path = '/content/models/'

N_subs = 32
N_trials = 40
N_chann = 32
fs = 128 #Hz
len_data = 7680# len without the initial seconds 8064
len_wind = 6 # 6 segs windows
N_windws =len_data/ (len_wind*fs) # 10 windows from 6 seconds


labels = 1 #(valence, arousal, dominance, liking)
f_bank = np.asarray([4., 40.])
lr = 1e-2
epochs = 500
verbose = 0

seed = 123
folds = 5

##save data

data_set = 'DEAP'
model_name = 'multiclass'
Experiment = 'cross_val'

In [12]:
t=time()

for sub in [0]:#range(N_subs):

    print('Subject: '+ str (sub+1))
    #-------- load subject
    with open(path_DEAP+'data_preprocessed_python/s'+ f"{sub+1:02}" + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
    X = subject['data'][:,0:N_chann,int(3*fs):]

    y_valance = (subject['labels'][:,0] > 5)*1
    y_arousal = (subject['labels'][:,1] > 5)*1

    y = []
    for i in range(len(y_valance)):
      if y_valance[i] == 1 and y_arousal[i] == 1:
        y.append(0)
      elif y_valance[i] == 0 and y_arousal[i] == 1:
        y.append(1)
      elif y_valance[i] == 0 and y_arousal[i] == 0:
        y.append(2)
      elif y_valance[i] == 1 and y_arousal[i] == 0:
        y.append(3)

    y = np.array(y)


    X_train = segmentation_trials(X, fs = fs, segs = len_wind ) # windowing 1 min trials in 6 seconds segments
    X_train = X_train[:,:,:,np.newaxis]


    y_train = np.repeat(y, N_windws) # reapiting layers 10 times
    num_class = np.unique(y_train).shape[0]
    Y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_class)
    print(X_train.shape, y_train.shape)



    #--------- load Network

    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=30, verbose=verbose, mode='min', min_delta=0.01, min_lr=0)
    terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [reduce_lr_on_plateau, terminate_on_nan]
    # build model
    clf = KerasClassifier(
        EEGNet,
        random_state=seed,

        #model hyperparameters
        nb_classes=num_class,
        Chans = X_train.shape[1],
        Samples = X_train.shape[2],
        kernLength = 128,
        F1 = 4,
        D = 4,
        F2 = 32,
        norm_rate = 0.5,
        dropoutType = 'Dropout',
        dropoutRate = 0.6,

        #model config
        verbose=1,
        callbacks=callbacks,
        batch_size=500, #full batch
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer="adam",
        optimizer_learning__rate=0.1,
        metrics = ['accuracy'],
        epochs = epochs
    )
    # search params
    param_grid =  {
                'F1':[4],
                'kernLength':[128],
                }

    #Gridsearch
    scoring = {"AUC": 'roc_auc', "Accuracy": make_scorer(accuracy_score),'Kappa':make_scorer(kappa)}

    cv = GridSearchCV(clf,param_grid,cv=StratifiedShuffleSplit(n_splits = folds, test_size = 0.2, random_state = seed),
                         verbose=1,n_jobs=1,
                         scoring=scoring,
                         refit="Accuracy",
                            )
    # frind best params with gridsearch
    cv.fit(X_train,Y_train)
    cv.cv_results_['best_index_']=cv.best_index_

    # best score
    print('Subject',sub+1,'Accuracy',cv.best_score_, 'std acc', cv.cv_results_['std_test_Accuracy'] ,'elapsed time',time()-t)
    print('---------')

    cv.best_estimator_.model_.save_weights(f'{save_models_path}Model_{Experiment}_{model_name}_sujeto_{sub+1}_'+data_set+'_weights.h5')
    with open(save_models_path+'Results_'+Experiment+'_'+model_name+'_sujeto_'+str(sub+1)+'_'+data_set+'.p','wb') as f:
        pickle.dump(cv.cv_results_,f)


Subject: 1
(400, 32, 768, 1) (400,)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 1.4625 - accuracy: 0.2906 - lr: 0.0010
Epoch 2/500
1/1 [==============================] - 0s 179ms/step - loss: 1.3795 - accuracy: 0.3094 - lr: 0.0010
Epoch 3/500
1/1 [==============================] - 0s 129ms/step - loss: 1.3815 - accuracy: 0.2406 - lr: 0.0010
Epoch 4/500
1/1 [==============================] - 0s 116ms/step - loss: 1.3680 - accuracy: 0.2906 - lr: 0.0010
Epoch 5/500
1/1 [==============================] - 0s 100ms/step - loss: 1.3586 - accuracy: 0.3313 - lr: 0.0010
Epoch 6/500
1/1 [==============================] - 0s 100ms/step - loss: 1.3407 - accuracy: 0.3344 - lr: 0.0010
Epoch 7/500
1/1 [==============================] - 0s 111ms/step - loss: 1.3554 - accuracy: 0.3156 - lr: 0.0010
Epoch 8/500
1/1 [==============================] - 0s 120ms/step - loss: 1.3588 - accuracy: 0.3313 - lr: 0.0010
Epoch 9/500


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/content/EEG_PianoRoll/functions/data_processing.py", line 23, in kappa
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))
NameError: name 'cohen_kappa_score' is not defined

  warnings.warn(


Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 1.3919 - accuracy: 0.3063 - lr: 0.0010
Epoch 2/500
1/1 [==============================] - 0s 166ms/step - loss: 1.3839 - accuracy: 0.2719 - lr: 0.0010
Epoch 3/500
1/1 [==============================] - 0s 128ms/step - loss: 1.3629 - accuracy: 0.3063 - lr: 0.0010
Epoch 4/500
1/1 [==============================] - 0s 105ms/step - loss: 1.3613 - accuracy: 0.3313 - lr: 0.0010
Epoch 5/500
1/1 [==============================] - 0s 99ms/step - loss: 1.3608 - accuracy: 0.2969 - lr: 0.0010
Epoch 6/500
1/1 [==============================] - 0s 102ms/step - loss: 1.3624 - accuracy: 0.3250 - lr: 0.0010
Epoch 7/500
1/1 [==============================] - 0s 101ms/step - loss: 1.3559 - accuracy: 0.3187 - lr: 0.0010
Epoch 8/500
1/1 [==============================] - 0s 98ms/step - loss: 1.3630 - accuracy: 0.3125 - lr: 0.0010
Epoch 9/500
1/1 [==============================] - 0s 101ms/step - loss: 1.3324 - accuracy: 0.3500 - lr: 0.00

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/content/EEG_PianoRoll/functions/data_processing.py", line 23, in kappa
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))
NameError: name 'cohen_kappa_score' is not defined

  warnings.warn(


Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 1.4146 - accuracy: 0.3313 - lr: 0.0010
Epoch 2/500
1/1 [==============================] - 0s 165ms/step - loss: 1.3775 - accuracy: 0.2781 - lr: 0.0010
Epoch 3/500
1/1 [==============================] - 0s 124ms/step - loss: 1.3534 - accuracy: 0.3406 - lr: 0.0010
Epoch 4/500
1/1 [==============================] - 0s 110ms/step - loss: 1.3580 - accuracy: 0.3094 - lr: 0.0010
Epoch 5/500
1/1 [==============================] - 0s 105ms/step - loss: 1.3493 - accuracy: 0.3313 - lr: 0.0010
Epoch 6/500
1/1 [==============================] - 0s 109ms/step - loss: 1.3653 - accuracy: 0.3063 - lr: 0.0010
Epoch 7/500
1/1 [==============================] - 0s 101ms/step - loss: 1.3512 - accuracy: 0.3125 - lr: 0.0010
Epoch 8/500
1/1 [==============================] - 0s 110ms/step - loss: 1.3528 - accuracy: 0.3125 - lr: 0.0010
Epoch 9/500
1/1 [==============================] - 0s 115ms/step - loss: 1.3472 - accuracy: 0.3344 - lr: 0.

...

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/content/EEG_PianoRoll/functions/data_processing.py", line 23, in kappa
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))
NameError: name 'cohen_kappa_score' is not defined

  warnings.warn(


Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 1.5099 - accuracy: 0.2375 - lr: 0.0010
Epoch 2/500
1/1 [==============================] - 0s 184ms/step - loss: 1.3842 - accuracy: 0.2844 - lr: 0.0010
Epoch 3/500
1/1 [==============================] - 0s 126ms/step - loss: 1.3744 - accuracy: 0.2875 - lr: 0.0010
Epoch 4/500
1/1 [==============================] - 0s 124ms/step - loss: 1.3593 - accuracy: 0.3187 - lr: 0.0010
Epoch 5/500
1/1 [==============================] - 0s 119ms/step - loss: 1.3465 - accuracy: 0.3406 - lr: 0.0010
Epoch 6/500
1/1 [==============================] - 0s 117ms/step - loss: 1.3542 - accuracy: 0.3531 - lr: 0.0010
Epoch 7/500
1/1 [==============================] - 0s 111ms/step - loss: 1.3450 - accuracy: 0.3500 - lr: 0.0010
Epoch 8/500
1/1 [==============================] - 0s 117ms/step - loss: 1.3602 - accuracy: 0.3031 - lr: 0.0010
Epoch 9/500
1/1 [==============================] - 0s 118ms/step - loss: 1.3434 - accuracy: 0.3375 - lr: 0.

1/1 [==============================] - 0s 36ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/content/EEG_PianoRoll/functions/data_processing.py", line 23, in kappa
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))
NameError: name 'cohen_kappa_score' is not defined

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 1.4761 - accuracy: 0.2750 - lr: 0.0010
Epoch 2/500
1/1 [==============================] - 0s 172ms/step - loss: 1.3840 - accuracy: 0.2675 - lr: 0.0010
Epoch 3/500
1/1 [==============================] - 0s 161ms/step - loss: 1.3648 - accuracy: 0.3025 - lr: 0.0010
Epoch 4/500
1/1 [==============================] - 0s 154ms/step - loss: 1.3394 - accuracy: 0.3525 - lr: 0.0010
Epoch 5/500
1/1 [==============================] - 0s 152ms/step - loss: 1.3760 - accuracy: 0.2600 - lr: 0.0010
Epoch 6/500
1/1 [==============================] - 0s 148ms/step - loss: 1.3652 - accuracy: 0.2925 - lr: 0.0010
Epoch 7/500
1/1 [==============================] - 0s 164ms/step - loss: 1.3507 - accuracy: 0.3350 - lr: 0.0010
Epoch 8/500
1/1 [==============================] - 0s 143ms/step - loss: 1.3638 - accuracy: 0.3125 - lr: 0.0010
Epoch 9/500
1/1 [==============================] - 0s 158ms/step - loss: 1.3393 - accuracy: 0.3275 - lr: 0.

# Accuracy Valence

In [ ]:
path_DEAP = '/content/drive/Shareddrives/GCPDS-Databases/DEAP/'
save_models_path = '/content/models/'

N_subs = 32
N_trials = 40
N_chann = 32
fs = 128 #Hz
len_data = 7680# len without the initial seconds 8064
len_wind = 6 # 6 segs windows
N_windws =len_data/ (len_wind*fs) # 10 windows from 6 seconds

labels = 0 #(0:valence, 1:arousal, dominance, liking)
f_bank = np.asarray([4., 40.])
lr = 1e-2
epochs = 500
verbose = 0

seed = 123
folds = 5

##save data

data_set = 'DEAP'
model_name = 'valence'
Experiment = 'cross_val'

In [ ]:
t=time()

for sub in [0]:#range(N_subs):

    print('Subject: '+ str (sub+1))
    #-------- load subject
    with open(path_DEAP+'data_preprocessed_python/s'+ f"{sub+1:02}" + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
    X = subject['data'][:,0:N_chann,int(3*fs):]
    y = (subject['labels'][:,labels] > 5)*1


    X_train = segmentation_trials(X, fs = fs, segs = len_wind )
    X_train = X_train[:,:,:,np.newaxis]

    y_train = np.repeat(y, N_windws)#tf.keras.utils.to_categorical(np.repeat(y, 4), num_classes=2)
    num_class = np.unique(y_train).shape[0]
    Y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_class)
    print(X_train.shape, y_train.shape)

    #--------- load Network

    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=30, verbose=verbose, mode='min', min_delta=0.01, min_lr=0)
    terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [reduce_lr_on_plateau, terminate_on_nan]
    # build model
    clf = KerasClassifier(
        EEGNet,
        random_state=seed,

        #model hyperparameters
        nb_classes=num_class,
        Chans = X_train.shape[1],
        Samples = X_train.shape[2],
        kernLength = 128,
        F1 = 4,
        D = 4,
        F2 = 32,
        norm_rate = 0.5,
        dropoutType = 'Dropout',
        dropoutRate = 0.6,

        #model config
        verbose=0,
        callbacks=callbacks,
        batch_size=500, #full batch
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer="adam",
        optimizer_learning__rate=0.1,
        metrics = ['accuracy'],
        epochs = epochs
    )
    # search params
    param_grid =  {
                'F1':[4],
                'kernLength':[128],
                }

    #Gridsearch
    scoring = {"AUC": 'roc_auc', "Accuracy": make_scorer(accuracy_score),'Kappa':make_scorer(kappa)}

    cv = GridSearchCV(clf,param_grid,cv=StratifiedShuffleSplit(n_splits = folds, test_size = 0.2, random_state = seed),
                         verbose=0,n_jobs=1,
                         scoring=scoring,
                         refit="Accuracy",
                            )
    # frind best params with gridsearch
    cv.fit(X_train,Y_train)
    cv.cv_results_['best_index_']=cv.best_index_

    # best score
    print('Subject',sub+1,'Accuracy',cv.best_score_, 'std acc', cv.cv_results_['std_test_Accuracy'] ,'elapsed time',time()-t)
    print('---------')

    cv.best_estimator_.model_.save_weights(f'{save_models_path}Model_{Experiment}_{model_name}_sujeto_{sub+1}_'+data_set+'_weights.h5')
    with open(save_models_path+'Results_'+Experiment+'_'+model_name+'_sujeto_'+str(sub+1)+'_'+data_set+'.p','wb') as f:
        pickle.dump(cv.cv_results_,f)